In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy


#importo los datos con np.loadtxt; el unpack=True "desempaca" las columnas en 2 arrays separados
Z,T = np.loadtxt("Datos.txt",unpack= True) 
hi=[]
D=[]
z=len(Z) 


#genero el vector de intervalos (el paso)
for i in range (1,z):
    hi.append(Z[i]-Z[i-1]) 
H=np.zeros([z,z])  #una matriz de 0s de dimension z*z


#aca se genera la primera matriz, que multiplicada por los coeficientes b (a_ix^3+b_ix^2+c_ix+d_i) va a resultar en los y_i (valores para este caso de T_i).
#para mas info consultar la teorica
for j in range(1,z-1): #TENER CUIDADO QUE EXISTEN LOS INDICES NEGATIVOS Y PUEDEN LLEGAR A TRAER PROBLEMAS
    H[j,j-1]=hi[j-1]  #va de la columna 0 a la z-2 (si hay 7 puntos va de la 0 a la 5); fila 1 a la z-1 (1-6) 
    H[j,j]=2*(hi[j-1]+hi[j]) #fila 1 a z-1 (1 a 6), columna 1 a z-1 (1 a 6)
    H[j,j+1]=hi[j] #fila 1 a z-1 (1 a 6), columna 2 a z (2 a 7)
    D.append(3*((T[j+1]-T[j])/hi[j] - (T[j]-T[j-1])/hi[j-1])) #genera los puntos del vector columna del otro lado de la igualdad
#se termina con una matriz H de dimension (z)*(z con y otra D, de dimension (z-2)*1. La idea es agregar condiciones de contorno (en este caso derivada segunda(=b)=0) en la primera y ultima posicion de D 
    
    
#agrega un 1 en la primera y ultima posicion
H[0,0]=1
H[z-1,z-1]=1 
#print(H)
D.insert(0,0) #inserta un 0 en el lugar 0
D.append(0) #inserta un 0 al final

#Con esto ya podemos resolver el sistema para calcular los coeficientes b_i 
b=np.linalg.solve(H,D) #resuelve el sistema

a=[]
c=[]
d=[]
#esto de los coeficientes creo que es innecesario pero no salia antes y quedo asi
coefa=[]
coefb=[]
coefc=[]
coefd=[]
# genera los coeficientes del polinomio a,c,d (ax^3+cx+d); el coeficiente b ya esta calculado anteriormente, nombrado como b. Los coeficientes a,c y d se calculan a partir de b
for i in range(0,z-1):
   # print(T[i])
    d.append(T[i])
    a.append((b[i+1]-b[i])/(3*hi[i]))
   # print(T[i+1],T[i],hi[i],b[i],a[i],hi[i]**2)    #print((T[i+1]-T[i])/hi[i],(b[i]/hi[i]),(a[i]*hi[i]**2))    #print(a)
    c.append(((T[i+1]-T[i])/hi[i])-(b[i]*hi[i])-(a[i]*hi[i]**2))
    #print(c) 
#print(d) #print(a) #print(c)
for i in range(z-1):
    coefa.append(a[i])
    coefb.append(b[i])
    coefc.append(c[i])
    coefd.append(d[i])

#crea una matriz con los coeficientes. Las columnas son a,b,c,d; las filas son los coeficientes del polinomio de cada intervalo.
P=(np.array(coefa),np.array(coefb),np.array(coefc),np.array(coefd))
P=np.array(P)
P=P.transpose()
#print(P) #print(len(P[:,0])) #print(P.shape)

#evaluacion de los polinomios y grafico
for i in range(len(P[:,0])):
    espaciado=np.linspace(Z[i],Z[i+1],20) #genera el espaciado entre la profundidad Z[i] y Z[i+1] con 20 puntos en el medio
    plt.plot(espaciado,np.polyval(P[i,:] , espaciado-Z[i])) #np.polyval utiliza todos los coeficientes ([i,:] agarra toda la fila i), y genera un polinomio. Espaciado -Z[i] lo que hace es mandar las coordenadas "x" a evaluar al 0. Sino estaria todo desfasado
#la sintaxis (plt.plot(espaciado,np.polyval(...))): espaciado genera el espaciado en x; el polyval evalua los polinomios y los plottea en el espaciado 
plt.plot(Z,T,"o") #esto marca los puntos que son dato

#una de las maneras de evaluar la derivada, haciendola analiticamente, multiplica por los exponentes a los coeficientes (ej: a_i^3-->3*a_i^2-->3*a[])
p1=copy.copy( P[:, :3] ) #copia las 3 primeras columnas (coeficientes a,b,c); el d no importa porque se esta evaluando la derivada
for i in range(3):
    p1[:,i]=(3-i)*p1[:,i]
Pol=[]
x=[]

#plotteo del grafico de la pendiente, mismo razonamiento que antes
for i in range(len(P[:,0])):
    espaciado=np.linspace(Z[i],Z[i+1],20)
    plt.plot(espaciado,np.polyval(p1[i,:], espaciado-Z[i]))
    y=np.polyval(p1[i,:], espaciado-Z[i])
    x.append(espaciado)
    Pol.append(y)
    #con los .append se estan guardando los datos de cada i para despues evaluar cual es el minimo y donde esta

#Evaluacion del minimo
v=min(np.array(Pol).ravel()) #el .ravel al final "desarma" el array y lo transforma en una figura 1-D (1 fila de n columnas); es necesario para poder evaluar el minimo y el argumento del minimo(su posicion) en una lista
m=np.argmin(np.array(Pol).ravel())
x=np.array(x).ravel()
print("La pendiente máxima se encuentra en:",x[m], "y su valor es de",v,"[°C/cm]")
J=0.01*v
print("El flujo máximo de calor es:", J, "[cal/s]")

OSError: Datos.txt not found.

El agua de un lago de zonas templadas puede dividirse en estratos térmicos. Cerca de la superficie el agua es
tibia y liviana (epilimnion) y en el fondo más fría y densa (hipolimnion). Lo mismo sucede en reactores
químicos. Ambas capas están separadas, aproximadamente, por un plano conocido por thermocline, donde la
derivada segunda de la temperatura respecto de la profundidad se hace cero (o la derivada primera tiene su
máximo). A esta profundidad el flujo de calor de la superficie al fondo de la capa se puede calcular con la ley de
Fourier, J=−k
dT
dz . Dados los datos de la tabla siguiente, correspondientes a la temperatura del líquido de
un reactor en función de la profundidad y usando el método de splines, realice un ajuste de la temperatura en
función de la profundidad y de su derivada. Encuentre la posición aproximada de la thermocline y calcule el
flujo de calor a través de la interfaz (tome k=0.01 cal(s. cmºC) )